In [1]:
import pandas as pd
from functools import reduce
import numpy as np
import math as math
from os import listdir

In [2]:
df1 = pd.read_csv("./csv/events_up_to_01062018.csv")

c:\python27\lib\site-packages\IPython\core\interactiveshell.py:2718: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df2 = pd.read_csv("./csv/labels_training_set.csv")

In [4]:
df1['timestamp'] = pd.to_datetime(df1['timestamp'])

In [5]:
def modelo(obj):
    if(pd.isnull(obj)):
        return obj
    
    list = obj.split(" ")
    if len(list) == 1:
        return obj
    if "iPhone" in list:
        return "iPhone"
    if "Samsung" in list:
        return "Samsung"
    if "Motorola" in list:
        return "Motorola"
    if "Lenovo" in list:
        return "Lenovo"
    if "Sony" in list:
        return "Sony"
    if "LG" in list:
        return "LG"
    if "Asus" in list:
        return "Asus"
    if "iPad" in list:
        return "iPad"
    if "Quantum" in list:
        return "Quantum"
    return obj

df1['model'] = df1['model'].apply(modelo)

In [6]:
df1['model'].value_counts()

iPhone              593603
Samsung             501307
Motorola            162914
LG                   24814
Sony                 15599
Lenovo               11382
iPad                  5663
Asus                  4536
Quantum               1693
Outros TV LED 15         1
Xiaomi Redmi 2           1
Name: model, dtype: int64

In [7]:
df1['event'].value_counts()

viewed product       1248124
brand listing         216312
visited site          204069
ad campaign hit       191388
generic listing       160176
searched products     130616
search engine hit     106406
checkout               65315
staticpage             11201
conversion              7091
lead                     983
Name: event, dtype: int64

# Analisis para los eventos distintos de convercion

In [8]:
df_a = df1

In [9]:
#calculo medianas de los modelos con eventos por persona
df_a_model = df_a.fillna(df1['model'].value_counts().idxmax()).loc[:,['person','model']].groupby('person')\
                .apply(lambda x: x['model'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"model_no_convercion"})


df_a_storage = df_a.fillna(df1['storage'].value_counts().idxmax()).loc[:,['person','storage']].groupby('person')\
                .apply(lambda x: x['storage'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"storage_no_convercion"})
        
        
df_a_condition = df_a.fillna(df1['condition'].value_counts().idxmax()).loc[:,['person','condition']].groupby('person')\
                .apply(lambda x: x['condition'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"condition_no_convercion"})
        
        
df_a_color = df_a.fillna(df1['color'].value_counts().idxmax()).loc[:,['person','color']].groupby('person')\
                .apply(lambda x: x['color'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"color_no_convercion"})

df_a_search_engine = df_a.fillna(df1['search_engine'].value_counts().idxmax()).loc[:,['person','search_engine']].groupby('person')\
                .apply(lambda x: x['search_engine'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"search_engine"})

In [10]:
df_datos_no_convercion = df_a_model.merge(df_a_storage,how='left',on='person')\
                                .merge(df_a_condition,how='left',on='person')\
                                .merge(df_a_color,how='left',on='person')\
                                .merge(df_a_search_engine,how='left',on='person')

In [11]:
dffinal = df_datos_no_convercion

# Date

# eventos por dia x usr ( al temporario)

In [12]:
# creo columnas con lunes == true o false
def dds(ele):# 0 -> lunes
    if(ele.month == 1):
        return (((ele.day + 6) % 7) == (i))
    if(ele.month == 2):
        return (((ele.day + 3 + 6) % 7) == i)
    if(ele.month == 3):
        return (((ele.day + 3 + 6) % 7) ==(i))
    if(ele.month == 4):
        return (((ele.day + 12) % 7) ==(i))
    if(ele.month == 5):
        return (((ele.day + 1 + 6) % 7) == (i))
i = 0
df1["lunes"] = df1["timestamp"].apply(dds)
i = 1
df1["martes"] = df1["timestamp"].apply(dds)
i = 2
df1["miercoles"] = df1["timestamp"].apply(dds)
i = 3
df1["jueves"] = df1["timestamp"].apply(dds)
i = 4
df1["viernes"] = df1["timestamp"].apply(dds)
i = 5
df1["sabado"] = df1["timestamp"].apply(dds)
i = 6
df1["domingo"] = df1["timestamp"].apply(dds)


In [13]:
dffinal = dffinal.set_index("person")

# mayor cantidad de eventos separados por dia de la semaan por usr 

In [14]:
# mas actividad x dia semana
def diaMasGrande(fila):
    aux = fila["lunes"]
    if(aux < fila["martes"]):
        aux = fila["martes"]
    if(aux < fila["miercoles"]):
        aux = fila["miercoles"]
    if(aux < fila["jueves"]):
        aux = fila["jueves"]
    if(aux < fila["viernes"]):
        aux = fila["viernes"]
    if(aux < fila["sabado"]):
        aux = fila["sabado"]
    if(aux < fila["domingo"]):
        aux = fila["domingo"]
    return aux
dffinal["numero mas usado dia"] = df1[["lunes","martes","miercoles","jueves","viernes","sabado","domingo","person"]].groupby("person").sum().apply(diaMasGrande,axis = 1)

# dia de la semana con mas eventos por usr

In [15]:
# dia de la semaan mas activa
def strDiaMasGrande(fila):
    strAux = "lunes"
    aux = fila["lunes"]
    if(aux < fila["martes"]):
        strAux = "martes"
        aux = fila["martes"]
    if(aux < fila["miercoles"]):
        strAux = "miercoles"
        aux = fila["miercoles"]
    if(aux < fila["jueves"]):
        strAux = "jueves"
        aux = fila["jueves"]
    if(aux < fila["viernes"]):
        strAux = "viernes"
        aux = fila["viernes"]
    if(aux < fila["sabado"]):
        strAux = "sabado"
        aux = fila["sabado"]
    if(aux < fila["domingo"]):
        strAux = "domingo"
        aux = fila["domingo"]
    return strAux
dffinal["nombre mas usado dia"] = df1[["lunes","martes","miercoles","jueves","viernes","sabado","domingo","person"]].groupby("person").sum().apply(strDiaMasGrande,axis = 1)

# cuento eventos por persona

In [16]:
aux = df1[["event","person"]]
aux["aux"] = 1
aux = aux.groupby(["person","event"]).count()
aux = aux.unstack().fillna(0).reset_index().reset_index()
aux.columns = aux.columns.droplevel()
aux["person"] = aux.iloc[:, 1].values
aux = aux.drop("",axis = 1)

c:\python27\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [17]:
dffinal = dffinal.reset_index()

In [18]:
aux = pd.merge(aux,dffinal,on="person",how="inner")

In [19]:
aux.columns

Index([u'ad campaign hit', u'brand listing', u'checkout', u'conversion',
       u'generic listing', u'lead', u'search engine hit', u'searched products',
       u'staticpage', u'viewed product', u'visited site', u'person',
       u'model_no_convercion', u'storage_no_convercion',
       u'condition_no_convercion', u'color_no_convercion', u'search_engine',
       u'numero mas usado dia', u'nombre mas usado dia'],
      dtype='object')

In [20]:
aux = aux.dropna(subset=["storage_no_convercion"])

In [21]:
aux["search_engine"] = aux["search_engine"].fillna("not")

In [22]:
aux["storage_no_convercion"] = aux["storage_no_convercion"].str.replace("GB","").replace("512MB","0.5")

# ultimo evento/fecha de actividad/de de la semana activo
ultimos evento y su fecha y dias de la semana

In [23]:
temp = df1[["person","timestamp"]].groupby("person").max().reset_index()
aux["ultima_fecha_de_actividad"] = temp["timestamp"]

In [24]:
diasDeLaSemana =["lunes","martes","miercoles","jueves","viernes","sabado","domingo"]
def dds2(ele):# 0 -> lunes
    if(ele.month == 1):
        for i in range(0,7):
            if(((ele.day + 6) % 7) == (i)):
                return diasDeLaSemana[i]  
    if(ele.month == 2):
        for i in range(0,7):
            if(((ele.day + 9) % 7) == (i)):
                return diasDeLaSemana[i]
    if(ele.month == 3):
        for i in range(0,7):
            if(((ele.day + 9) % 7) == (i)):
                return diasDeLaSemana[i]
    if(ele.month == 4):
        for i in range(0,7):
            if(((ele.day + 12) % 7) == (i)):
                return diasDeLaSemana[i]
    if(ele.month == 5):
        for i in range(0,7):
            if(((ele.day) % 7) == (i)):
                return diasDeLaSemana[i]
            
aux["ultimo_dia_de_la_semana_activo"] = aux["ultima_fecha_de_actividad"].apply(dds2)

In [25]:
temp = df1[["person","timestamp"]].groupby("person")["timestamp"].max().reset_index()

In [26]:
df2 = pd.merge( df1[["person","timestamp","event"]],temp,on=["person","timestamp"],how="right")
#personas con varios eventos en el mismo timestamp/ como elegir cual evento fue el ultimo? 
#-->uso moda x q no se me ocurre nada mejor
tmpmod = df2.groupby("person").agg(pd.Series.mode)

In [27]:
aux["ultimo_evento_registrado_cmode"] = tmpmod.reset_index()["event"]

In [28]:
aux.columns

Index([u'ad campaign hit', u'brand listing', u'checkout', u'conversion',
       u'generic listing', u'lead', u'search engine hit', u'searched products',
       u'staticpage', u'viewed product', u'visited site', u'person',
       u'model_no_convercion', u'storage_no_convercion',
       u'condition_no_convercion', u'color_no_convercion', u'search_engine',
       u'numero mas usado dia', u'nombre mas usado dia',
       u'ultima_fecha_de_actividad', u'ultimo_dia_de_la_semana_activo',
       u'ultimo_evento_registrado_cmode'],
      dtype='object')

In [29]:
temp = pd.to_datetime("06.01.2018") - aux["ultima_fecha_de_actividad"]

In [30]:
#lo paso a dias hasta el 1 / 06
aux["dias_hasta"] = temp.reset_index()["ultima_fecha_de_actividad"].dt.days

In [31]:
word = df1["event"].unique()
word

array(['viewed product', 'searched products', 'ad campaign hit',
       'staticpage', 'checkout', 'search engine hit', 'conversion',
       'generic listing', 'brand listing', 'visited site', 'lead'], dtype=object)

In [32]:
# fecha ultimo evento
ultimosEveFecha = df1[["person","timestamp"]].groupby("person")["timestamp"].max().reset_index().set_index("person")
del ultimosEveFecha["timestamp"]

In [33]:
eventList = df1["event"].unique()
for word in eventList:
    ultimosEveFecha[word + "_last"] = df1.loc[df1["event"] == word][["person","timestamp"]].groupby("person").max()

In [34]:
for word in eventList:
    ultimosEveFecha[word + "_last"] = (pd.to_datetime("06.01.2018") - pd.to_datetime(ultimosEveFecha[word + "_last"])).dt.days
    ultimosEveFecha[word + "_last"] = ultimosEveFecha[word + "_last"].fillna(ultimosEveFecha[word + "_last"].mode()[0])

In [35]:

ultimosEveFecha = ultimosEveFecha.reset_index()

In [36]:
aux = pd.merge(aux,ultimosEveFecha,on="person",how="left")

In [37]:
aux.isna().sum()

ad campaign hit                   0
brand listing                     0
checkout                          0
conversion                        0
generic listing                   0
lead                              0
search engine hit                 0
searched products                 0
staticpage                        0
viewed product                    0
visited site                      0
person                            0
model_no_convercion               0
storage_no_convercion             0
condition_no_convercion           0
color_no_convercion               0
search_engine                     0
numero mas usado dia              0
nombre mas usado dia              0
ultima_fecha_de_actividad         0
ultimo_dia_de_la_semana_activo    0
ultimo_evento_registrado_cmode    0
dias_hasta                        0
viewed product_last               0
searched products_last            0
ad campaign hit_last              0
staticpage_last                   0
checkout_last               

In [38]:
#cantidad de dias en los q entro
temp = (pd.to_datetime("06.01.2018") - df1["timestamp"])
df1["dias_hasta_1/6"] = temp.dt.days
df1["horas_hasta_1/6"] = temp.dt.seconds / 3600 +  temp.dt.days * 24

In [39]:
aux = aux.set_index("person")
aux["cantidad_de_dias_utilizado"] = df1[["person","dias_hasta_1/6"]].groupby("person")["dias_hasta_1/6"].nunique().reset_index().set_index("person")
aux = aux.reset_index()

In [40]:
eventList

array(['viewed product', 'searched products', 'ad campaign hit',
       'staticpage', 'checkout', 'search engine hit', 'conversion',
       'generic listing', 'brand listing', 'visited site', 'lead'], dtype=object)

In [41]:
def concatenador(item):
       return reduce(lambda x, y: x + y, item)

In [42]:
#def enlistar(x):
#    return [x]
#df1["dias_hasta_1/6"] = df1["dias_hasta_1/6"].apply(enlistar)

In [43]:
#aux["fechas_de_conversiones"] = df1[["event","person","dias_hasta_1/6"]].loc[df1["event"] == "conversion"].groupby("person")["dias_hasta_1/6"].agg(concatenador).reset_index()[0]

In [44]:
#aux["fechas_de_busqueda"] = df1[["event","person","dias_hasta_1/6"]].loc[df1["event"] == "searched products"].groupby("person")["dias_hasta_1/6"].agg(concatenador).reset_index()[0]

In [45]:
df1["dia_de_la_semana"]= df1["timestamp"].dt.dayofweek

In [46]:
#aux["media_dia_de_la_semana_convercion"] = df1.loc[df1.event == "conversion",["person","dia_de_la_semana"]].groupby('person')\
 #       .apply(lambda x: x["dia_de_la_semana"].value_counts().idxmax()).reset_index()[0]

In [47]:
aux["media_dia_de_la_semana_no_convercion"] = df1.loc[df1.event != "conversion",["person","dia_de_la_semana"]].groupby('person')\
        .apply(lambda x: x["dia_de_la_semana"].value_counts().idxmax()).reset_index()[0]

In [48]:
temp = df1[["dias_hasta_1/6","person"]].groupby("person").apply(lambda x: x["dias_hasta_1/6"].max() - x["dias_hasta_1/6"].min()).reset_index() 

aux = pd.merge(aux,temp,on="person",how="left").rename(index=str, columns={0: "delta_ultimo_primer_dia_actividad"})

In [49]:
temp = df1[["dias_hasta_1/6","person"]].groupby("person").apply(lambda x: x["dias_hasta_1/6"].max() - x["dias_hasta_1/6"].nlargest(2).values[-1]).reset_index()

aux = pd.merge(aux,temp,on="person",how="left").rename(index=str, columns={0: "delta_ultimo_anteulit_tiempo_actividad"})

In [50]:
temp = df1[["dias_hasta_1/6","person"]].groupby("person").apply(lambda x: x["dias_hasta_1/6"].nsmallest(2).values[-1] - x["dias_hasta_1/6"].min()).reset_index()

aux = pd.merge(aux,temp,on="person",how="left").rename(index=str, columns={0: "delta_primer_seg_tiempo_actividad"})

In [51]:
df1 = df1.sort_values(by=['person', 'dias_hasta_1/6'])


In [52]:
temp =  df1[["dias_hasta_1/6","person"]].groupby('person').diff()
df1["dTimeEve0"] = temp.fillna(0)
df1["dTimeEve1"] = temp.fillna(1)

In [53]:
temp = df1[["person","dTimeEve0"]].groupby('person').max().reset_index()
aux = pd.merge(aux,temp,on="person",how="left")
aux = aux.rename(columns={"dTimeEve0":"dTimeEveMax"})

In [54]:
temp = df1[["person","dTimeEve0"]].groupby('person').mean().reset_index()
aux = pd.merge(aux,temp,on="person",how="left")
aux = aux.rename(columns={"dTimeEve0":"dTimeEveMean0"})

In [55]:
temp = df1[["person","dTimeEve1"]].groupby('person').mean().reset_index()
aux = pd.merge(aux,temp,on="person",how="left")
aux = aux.rename(columns={"dTimeEve1":"dTimeEveMean1"})

In [56]:
temp =  df1[["horas_hasta_1/6","person"]].groupby('person').diff()
df1["dHTimeEve0"] = temp.fillna(0)
df1["dHTimeEve1"] = temp.fillna(1)

In [57]:
temp = df1[["person","dHTimeEve0"]].groupby('person').max().reset_index()
aux = pd.merge(aux,temp,on="person",how="left")
aux = aux.rename(columns={"dHTimeEve0":"dHTimeEveMax"})

In [58]:
temp = df1[["person","dHTimeEve1"]].groupby('person').mean().reset_index()
aux = pd.merge(aux,temp,on="person",how="left")
aux = aux.rename(columns={"dHTimeEve1":"dHTimeEveMean1"})

In [59]:
temp = df1[["person","dHTimeEve0"]].groupby('person').mean().reset_index()
aux = pd.merge(aux,temp,on="person",how="left")
aux = aux.rename(columns={"dHTimeEve0":"dHTimeEveMean0"})

In [60]:
del aux["ultima_fecha_de_actividad"]
#aux.to_csv("features.csv",index = False)

# Cuento cantidad de eventos por modelo por persona

In [61]:
df1.condition.unique()

array(['Bom', 'Muito Bom', nan, 'Excelente', 'Novo', 'Bom - Sem Touch ID'], dtype=object)

In [62]:
df1['condition'].value_counts()

Bom                   547617
Excelente             378697
Muito Bom             359053
Bom - Sem Touch ID     31742
Novo                    3421
Name: condition, dtype: int64

In [63]:
def condicion(obj):
    if(pd.isnull(obj)):
        return obj
    
    obj = obj.replace("Excelente","Estado Excelente")
    obj = obj.replace("Muito Bom","Estado Muy Bien")
    obj = obj.replace("Novo","Estado Nuevo")
    obj = obj.replace("Bom - Sem Touch ID","Estado Bien") #No se que quise encriptar Trofcafone con este cifrado
                                                          #Asi que hare de cuenta que no existe
    obj = obj.replace("Bom","Estado Bien")
    return obj
    
df1["condition"] = df1["condition"].apply(condicion)

In [64]:
df1.condition.unique()

array(['Estado Bien', 'Estado Muy Bien', nan, 'Estado Excelente',
       'Estado Nuevo'], dtype=object)

In [65]:
df1['condition'].value_counts()

Estado Bien         579359
Estado Excelente    378697
Estado Muy Bien     359053
Estado Nuevo          3421
Name: condition, dtype: int64

In [66]:
auxi = df1[["condition","person"]]
auxi["cont"] = 1
auxi.head()

c:\python27\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,condition,person,cont
1505383,Estado Bien,0008ed71,1
1507286,Estado Muy Bien,0008ed71,1
1507716,Estado Bien,0008ed71,1
2122051,NaN,0008ed71,1
2336760,NaN,0008ed71,1


In [67]:
auxi.columns

Index([u'condition', u'person', u'cont'], dtype='object')

In [68]:
auxi = auxi.groupby(['person','condition']).agg({'cont':sum})
auxi.head()

cont
person   condition             
0008ed71 Estado Bien          2
         Estado Muy Bien      1
00091926 Estado Bien        102
         Estado Excelente   108
         Estado Muy Bien    163

In [69]:
auxi_listo = auxi.unstack(fill_value=0)
auxi_listo.head()

cont                                              
condition Estado Bien Estado Excelente Estado Muy Bien Estado Nuevo
person                                                             
0008ed71            2                0               1            0
00091926          102              108             163            1
00091a7a            1                2               0            0
000ba417          115               11              34            0
000c79fe            4                0               0            0

In [70]:
auxi_listo.columns = auxi_listo.columns.droplevel(0)
auxi_listo.head()

condition,Estado Bien,Estado Excelente,Estado Muy Bien,Estado Nuevo
person,,,,
0008ed71,2,0,1,0
00091926,102,108,163,1
00091a7a,1,2,0,0
000ba417,115,11,34,0
000c79fe,4,0,0,0


In [71]:
auxi_listo = auxi_listo.reset_index(level='person')
auxi_listo.head()

condition,person,Estado Bien,Estado Excelente,Estado Muy Bien,Estado Nuevo
0,0008ed71,2,0,1,0
1,00091926,102,108,163,1
2,00091a7a,1,2,0,0
3,000ba417,115,11,34,0
4,000c79fe,4,0,0,0


In [72]:
aux = aux.merge(auxi_listo, left_on = 'person', right_on = 'person', how = 'right')

In [73]:
aux.head()

,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,...,dTimeEveMax,dTimeEveMean0,dTimeEveMean1,dHTimeEveMax,dHTimeEveMean1,dHTimeEveMean0,Estado Bien,Estado Excelente,Estado Muy Bien,Estado Nuevo
0,0008ed71,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.166667,4.013889,0.185324,0.018657,2,0,1,0
1,00091926,15.0,25.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.062500,0.064732,74.207500,1.497148,1.494916,102,108,163,1
2,00091a7a,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.100000,0.096389,0.108722,0.008722,1,2,0,0
3,000ba417,1.0,24.0,6.0,1.0,14.0,0.0,1.0,0.0,0.0,...,7.0,0.043689,0.048544,172.998056,1.026614,1.021760,115,11,34,0
4,000c79fe,1.0,0.0,1.0,0.0,1.0,0.0,1.0,9.0,0.0,...,0.0,0.000000,0.058824,0.172222,0.066667,0.007843,4,0,0,0


In [74]:
#auxilio.to_csv("features.csv",index = False)

In [75]:
df1['new_vs_returning'].value_counts()

Returning    165827
New           38242
Name: new_vs_returning, dtype: int64

In [77]:
auxi_augusto = df1.loc[:,["person","new_vs_returning"]]
auxi_augusto['contador_visitas_ret_new'] = 1
auxi_augusto = auxi_augusto.dropna().reset_index()

In [78]:
auxi_augusto.head()

,index,person,new_vs_returning,contador_visitas_ret_new
0,2336760,0008ed71,New,1
1,2336761,0008ed71,Returning,1
2,2146952,00091926,Returning,1
3,2146953,00091926,Returning,1
4,2146951,00091926,Returning,1


In [79]:
auxi_augusto = auxi_augusto.groupby(['person']).agg({'contador_visitas_ret_new':sum})
auxi_augusto.head()
auxi_augusto = auxi_augusto.reset_index(level='person')


In [80]:
#hay que ver new vs returning en cuanto distancia de tiempo
aux = aux.merge(auxi_augusto, left_on = 'person', right_on = 'person', how = 'right')
aux.fillna(1, inplace = True)
aux.head()

,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,...,dTimeEveMean0,dTimeEveMean1,dHTimeEveMax,dHTimeEveMean1,dHTimeEveMean0,Estado Bien,Estado Excelente,Estado Muy Bien,Estado Nuevo,contador_visitas_ret_new
0,0008ed71,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.000000,0.166667,4.013889,0.185324,0.018657,2.0,0.0,1.0,0.0,2
1,00091926,15.0,25.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.062500,0.064732,74.207500,1.497148,1.494916,102.0,108.0,163.0,1.0,34
2,00091a7a,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.100000,0.096389,0.108722,0.008722,1.0,2.0,0.0,0.0,1
3,000ba417,1.0,24.0,6.0,1.0,14.0,0.0,1.0,0.0,0.0,...,0.043689,0.048544,172.998056,1.026614,1.021760,115.0,11.0,34.0,0.0,6
4,000c79fe,1.0,0.0,1.0,0.0,1.0,0.0,1.0,9.0,0.0,...,0.000000,0.058824,0.172222,0.066667,0.007843,4.0,0.0,0.0,0.0,1


In [81]:
temp = df1.loc[(~(df1["condition"].isna()))&(~(df1["dias_hasta_1/6"].isna()))][["person","dias_hasta_1/6","condition"]]\
                .groupby("person").apply(lambda x: x.loc[x["dias_hasta_1/6"].idxmax(),["dias_hasta_1/6","condition"]])\
                .reset_index()[["person","condition"]].rename(columns={"condition":"condition_last"})
aux = pd.merge(aux,temp,on="person",how ="left")

In [82]:
temp = df1.loc[(~(df1["condition"].isna()))&(~(df1["dias_hasta_1/6"].isna()))][["person","dias_hasta_1/6","condition"]]\
                .groupby("person").apply(lambda x: x.loc[x["dias_hasta_1/6"].idxmin(),["dias_hasta_1/6","condition"]])\
                .reset_index()[["person","condition"]].rename(columns={"condition":"condition_first"})
aux = pd.merge(aux,temp,on="person",how ="left")

In [83]:
temp = df1.loc[(~(df1["condition"].isna()))&(~(df1["dias_hasta_1/6"].isna()))][["person","dias_hasta_1/6","condition"]]\
                .groupby("person").apply(lambda x: x.loc[x["dias_hasta_1/6"].median() == x["dias_hasta_1/6"],["dias_hasta_1/6","condition"]].head(1))\
                .reset_index()[["person","condition"]].rename(columns={"condition":"condition_med"})

aux = pd.merge(aux,temp,on="person",how ="left")

In [84]:
aux.head()

,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,...,dHTimeEveMean1,dHTimeEveMean0,Estado Bien,Estado Excelente,Estado Muy Bien,Estado Nuevo,contador_visitas_ret_new,condition_last,condition_first,condition_med
0,0008ed71,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.185324,0.018657,2.0,0.0,1.0,0.0,2,Estado Bien,Estado Bien,Estado Bien
1,00091926,15.0,25.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.497148,1.494916,102.0,108.0,163.0,1.0,34,Estado Nuevo,Estado Muy Bien,Estado Muy Bien
2,00091a7a,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.108722,0.008722,1.0,2.0,0.0,0.0,1,Estado Bien,Estado Bien,Estado Bien
3,000ba417,1.0,24.0,6.0,1.0,14.0,0.0,1.0,0.0,0.0,...,1.026614,1.021760,115.0,11.0,34.0,0.0,6,Estado Excelente,Estado Bien,Estado Bien
4,000c79fe,1.0,0.0,1.0,0.0,1.0,0.0,1.0,9.0,0.0,...,0.066667,0.007843,4.0,0.0,0.0,0.0,1,Estado Bien,Estado Bien,Estado Bien


In [85]:
def fun(serie):
    if(pd.isnull(serie)):
        return serie
    lista = serie.split(" ")
    return lista[0]

In [86]:
aux_sistema_op = df1.loc[:,['person','operating_system_version']]
aux_sistema_op['operating_system_version']= aux_sistema_op['operating_system_version'].apply(fun)

In [88]:
grupo = aux_sistema_op.fillna(aux_sistema_op['operating_system_version'].value_counts().idxmax()).groupby('person').\
                    apply(lambda x: x['operating_system_version'].value_counts().idxmax())\
                    .reset_index(level='person').\
                    rename(index=str, columns={0: "sistema_operativo"})

In [89]:
aux = aux.merge(grupo,left_on = 'person', right_on = 'person', how = 'left')

In [90]:
aux.to_csv("features.csv",index = False)

In [91]:
aux.columns

Index([u'person', u'ad campaign hit', u'brand listing', u'checkout',
       u'conversion', u'generic listing', u'lead', u'search engine hit',
       u'searched products', u'staticpage', u'viewed product', u'visited site',
       u'model_no_convercion', u'storage_no_convercion',
       u'condition_no_convercion', u'color_no_convercion', u'search_engine',
       u'numero mas usado dia', u'nombre mas usado dia',
       u'ultimo_dia_de_la_semana_activo', u'ultimo_evento_registrado_cmode',
       u'dias_hasta', u'viewed product_last', u'searched products_last',
       u'ad campaign hit_last', u'staticpage_last', u'checkout_last',
       u'search engine hit_last', u'conversion_last', u'generic listing_last',
       u'brand listing_last', u'visited site_last', u'lead_last',
       u'cantidad_de_dias_utilizado', u'media_dia_de_la_semana_no_convercion',
       u'delta_ultimo_primer_dia_actividad',
       u'delta_ultimo_anteulit_tiempo_actividad',
       u'delta_primer_seg_tiempo_actividad', u'd